In [15]:
import os
import sys
from scipy.integrate import solve_ivp
import numpy as np
from numpy.linalg import norm
from numpy import sqrt
from math import pi
import pickle
import matplotlib.pyplot as plt
from packages import data_container
from packages.data_container import Data
from packages.helper import play_trajs, rotate, sp2a, v2sp, dist, psi, beta, d_theta, d_psi
# For pickle to load the Data object, which is defined in packages.data_container
sys.modules['data_container'] = data_container


In [11]:
file = os.path.abspath(os.path.join(os.getcwd(), os.pardir, 'Raw Data', 'Bai_movObst1_data.pickle'))
with open(file, 'rb') as f:
    data = pickle.load(f)

In [69]:
set(data.info['obst_angle'])
# set(data.info['obst_speed'])

{-180.0, -157.5, -135.0, -112.5, -90.0, 0.0, 90.0, 112.5, 135.0, 157.5, 180.0}

In [39]:
'''Animate data'''
%matplotlib qt
i = 1
# p_obst = np.array(data.info['p_obst'][i])
t0 = data.info['stimuli_onset'][i]
p_goal = np.array(data.info['p_goal'][i])
p_subj = np.array(data.get_traj(i))[:,[0,2]]
p_obst = np.array(data.info['p_obst'][i])
trajs = [p_goal, p_obst, p_subj]
ws = [data.info['w_goal'], data.info['w_obst'], 0.4]
play_trajs(trajs, ws, data.Hz)

In [120]:
'''Plot data by condition'''
%matplotlib qt
plt.figure()
con_ang = [180]
con_spd = [1.3]
for i in range(len(data.trajs)):
    obst_speed = data.info['obst_speed'][i]
    obst_angle = data.info['obst_angle'][i]
    if not (obst_speed in con_spd and abs(obst_angle) in con_ang):
        continue
    subj = np.array(data.get_traj(i))[:, [0,2]]
    obst = np.array(data.info['p_obst'][i])
    if i % 2 == 0:
        subj = rotate(subj, np.arctan(11 / 9) - pi / 2)
        obst = rotate(obst, np.arctan(11 / 9) - pi / 2)
    else:
        subj = rotate(subj, np.arctan(11 / 9) + pi / 2)
        obst = rotate(obst, np.arctan(11 / 9) + pi / 2)
    if obst_angle < 0:
        subj[:, 0] *= -1
        obst[:, 0] *= -1
    plt.plot(subj[:, 0], subj[:, 1])
    plt.plot(obst[:, 0], obst[:, 1])
ax = plt.gca()
ax.set_aspect('equal')
ax.set_title('angle: ' + str(con_ang[0]) + ' speed: ' + str(con_spd[0]))

Text(0.5, 1.0, 'angle: 180 speed: 1.3')

In [22]:
'''ode45 simuation toy example'''
def funcs(t, w, models):
    def linear(w, a):
        x, v = w
        dwdt = [v, a]
        return dwdt
    
    def linear2(w, a):
        x, v = w
        dwdt = [v, a]
        return dwdt
    
    dwdt = np.zeros_like(w)
    # Set first order derivative
    dwdt[0] = w[1]
    
    # Set second order derivative
    for model in models:
        if model['name'] == 'linear':
            a = model['a']
            dwdt[1] += linear(w, a)[1]
        elif model['name'] == 'linear2':
            a = model['a']
            dwdt[1] += linear2(w, a)[1]

    return dwdt
w0 = [0, 0]
model1 = {'name': 'linear', 'a': 2}
model2 = {'name': 'linear2', 'a': -1}
models = [model1, model2]
sol = solve_ivp(funcs, [0, 10], [0, 0], t_eval=[0, 1, 2, 3], args=[models])
sol.y

array([[0. , 0.5, 2. , 4.5],
       [0. , 1. , 2. , 3. ]])

In [54]:
'''ode45 simulation sp version'''
def ode_func(t, var, models, args):
    xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi = var
    ref, w_goal, w_obst = args['ref'], args['w_goal'], args['w_obst']
    r_g = dist([x, y], [xg, yg])
    psi_g = psi([x, y], [xg, yg], ref=ref)
    beta_o = beta([x, y], [xo, yo], [vx, vy])
    d_theta_o = d_theta([x, y], [xo, yo], [vx, vy], [vxo, vyo], w_obst)
    d_psi_o = d_psi([x, y], [xo, yo], [vx, vy], [vxo, vyo])
    sigmoid = 1 / (1 + np.exp(20 * (np.absolute(beta_o) - 1.3)))

    def Fajen_approach(args):    
        ps, b1, k1, c1, c2, k2 = args['ps'], args['b1'], args['k1'], args['c1'], args['c2'], args['k2']
        ddphi = -b1 * dphi - k1 * (phi - psi_g) * (np.exp(-c1 * r_g) + c2)
        ds = k2 * (ps - s)
        ax, ay = sp2a(s, ds, phi, dphi, ref=ref)
        output = [ax, ay, ds, ddphi]
        return output
    
    # Known issue: When d_psi_o is zero, it becomes a null model.
    def Cohen_avoid2(args):
        ps, b1, k1, c5, c6, b2, k2, c7, c8 = args['ps'], args['b1'], args['k1'], args['c5'], args['c6'], args['b2'], args['k2'], args['c7'], args['c8']
        ddphi = -b1 * dphi - d_psi_o * k1 * np.exp(-c5 * np.absolute(d_psi_o)) * (1 - np.exp(-c6 * np.maximum(0, d_theta_o))) * sigmoid
        ds = b2 * (ps - s) + d_psi_o * k2 * np.exp(-c7 * np.absolute(d_psi_o)) * (1 - np.exp(-c8 * np.maximum(0, d_theta_o))) * sigmoid
        ax, ay = sp2a(s, ds, phi, dphi, ref=ref)
        output = [ax, ay, ds, ddphi]
        return output
    
    output = np.array([0.0] * 4)
    for model in models:
        if model['name'] == 'Cohen_avoid2':
            output += np.array(Cohen_avoid2(model))
        elif model['name'] == 'Fajen_approach':
            output += np.array(Fajen_approach(model))
    # xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi = var
    dvardt = [0, 0, vxo, vyo, 0, 0, vx, vy, output[0], output[1], dphi, output[2], output[3]]
    return dvardt

xg0, yg0, xo0, yo0, x0, y0 = 0, 10, 1, 10, 0, 0
vxo0, vyo0, vx0, vy0 = 0, -1, 0, 1
s0, phi0 = v2sp([vx0, vy0])
dphi0 = 0
var0 = [xg0, yg0, xo0, yo0, vxo0, vyo0, x0, y0, vx0, vy0, phi0, s0, dphi0]
Fajen_approach = {'name': 'Fajen_approach', 'ps': 1, 'b1': 3.25, 'k1': 7.5, 'c1': 0.4, 'c2': 0.4, 'k2': 1.4}
Cohen_avoid2 = {'name': 'Cohen_avoid2', 'ps': 1, 'b1': 2.62, 'k1': 375, 'c5': 11, 'c6': 0.87, 'b2': -0.68, 'k2': -476, 'c7': 57.84, 'c8': 18.18}
# TODO test only approach model (compare with old simulator) and two models combined, then add visual var to avoid model
models = [Fajen_approach, Cohen_avoid2]
Hz = 100
t_total = 10
t_eval = np.linspace(0, t_total, t_total * Hz)
args = {'ref': [0,  1], 'w_goal': 0.1, 'w_obst': 0.1}
sol = solve_ivp(ode_func, [0, 10], var0, t_eval=t_eval, args=[models, args])
xg, yg, xo, yo, vxo, vyo, x, y, vx, vy, phi, s, dphi = sol.y

In [55]:
%matplotlib qt
p_goal = np.stack((xg, yg), axis=-1)
p_obst = np.stack((xo, yo), axis=-1)
p_model = np.stack((x, y), axis=-1)
ws = [.1, 0.1, 0.4]
Hz = 90
play_trajs([p_goal, p_obst, p_model], ws, Hz)